In [1]:
import numpy as np
import celmech as cm
import rebound as rb
import sympy as sp

import sys
sys.path.append("../code/")

In [2]:
# Generate a simulation of equal mass, evenly spaced planets.
def get_sim(m,Npl,pratio,exfrac):
    alpha = pratio**(-2/3)
    ex = np.min((1/alpha-1,1-alpha)) # orbit-crossing eccentricity
    ecc = exfrac * ex
    sim = rb.Simulation()
    sim.add(m=1)
    P = 1
    for i in range(Npl):
        sim.add(m=m,P=P,l=np.random.uniform(-np.pi,np.pi),e=ecc,pomega= np.random.uniform(-np.pi,np.pi))
        P*= pratio
    sim.move_to_com()
    return sim

In [3]:
pratio = 4/3
sim = get_sim(1e-5,5,pratio,0.2) # system of five planets

pvars = cm.Poincare.from_Simulation(sim)
pham = cm.PoincareHamiltonian(pvars)
Hkep = pham.H.copy()

pham_inits = pham.state.values

jlo,jhi = np.floor(1 + 1/(pratio-1)),np.ceil(1 + 1/(pratio-1))
jlo,jhi = int(jlo),int(jhi)

Add MMRs to celmech model

In [4]:
Npl = pham.N - 1
for i in range(1,Npl):
    pham.add_MMR_terms(jlo,1,max_order = 2 , indexIn=i, indexOut=i+1,inclinations=False)
    

Express $\lambda$ variables in terms of resonant combinations

In [5]:
lmbda_vec = [pham.qp_vars[3*i] for i in range(Npl)]

T = 2 * sp.Matrix([
    [1-jlo,jlo,0,0,0],
    [0,1-jlo,jlo,0,0],
    [0,0,1-jlo,jlo,0],
    [0,0,0,1-jlo,jlo],
    [1,-1,0,0,0]
])
angvec = [*sp.symbols("phi(1:5)")] + [sp.symbols("psi")]
angrule = dict(zip(lmbda_vec,T.inv() * sp.Matrix(angvec)))

Get symbolic expressions for $A, \mathbf{b}$ from celmech model

In [6]:
xvec = [pham.qp_vars[3*i+1] for i in range(Npl)] + [pham.qp_vars[Npl * 3 + 3*i+1] for i in range(Npl)]
xvec

[η₁, η₂, η₃, η₄, η₅, κ₁, κ₂, κ₃, κ₄, κ₅]

In [7]:
phi2=sp.symbols("phi2",real=True)
A_mtrx=sp.Matrix(2*Npl,2*Npl, lambda i,j:sp.diff(pham.N_H,xvec[i],xvec[j]))
A_mtrx = A_mtrx.xreplace(angrule)

zero_rule = dict(zip(xvec,np.zeros(2*Npl)))
b_vec = [sp.diff(pham.N_H,xvec[i]).xreplace(zero_rule).xreplace(angrule) for i in range(2*Npl)]

Initialize resonant interaction objects

In [8]:
from ResonantInteraction import EccentricityResonanceInteraction

interactions = []
for i in range(1,Npl):
    interaction = EccentricityResonanceInteraction(pvars,i,i+1,jlo * 2)
    interactions.append(interaction)

Choose some random resonant angles

In [9]:
np.random.seed(123)
phi_vals = np.random.uniform(-np.pi,np.pi,size=4)

Evaluate the full $A$ matrix and $\mathbf{b}$ vector for the system from a sum over pair-wise interactions

In [10]:
A = np.zeros((2*Npl,2*Npl))
b = np.zeros(2*Npl)
for i,interaction in enumerate(interactions):
    j=i+1
    phi = phi_vals[i]
    s,c = np.sin(phi/2),np.cos(phi/2)
    s2 = 2 * s * c
    c2 = c*c - s*s
    Aij = interaction.A(c2,s2)
    bij = interaction.b(c,s)
    b[[i,j,i+Npl,j+Npl]] += bij
    A[i:j+1,i:j+1] += Aij[:2,:2]
    A[i+Npl:j+Npl+1,i:j+1] += Aij[2:,:2]
    A[i:j+1,i+Npl:j+Npl+1] += Aij[:2,2:]
    A[i+Npl:j+Npl+1,i+Npl:j+Npl+1] += Aij[2:,2:]
    
    

$A$ is a block-tridiagonal of the form
$$
A = \begin{pmatrix}
 A_1 & A_2 \\
 A_2 & - A_1
 \end{pmatrix}
$$
where $A_1$ and $A_2$ are tri-diagonal:

In [16]:
print(np.round(1e4*A,1))

[[  3.7  -4.2   0.    0.    0.   10.7 -11.9   0.    0.    0. ]
 [ -4.2   6.5  -2.1   0.    0.  -11.9   5.    9.2   0.    0. ]
 [  0.   -2.1   1.4   1.    0.    0.    9.2 -16.5   7.    0. ]
 [  0.    0.    1.    3.4  -5.    0.    0.    7.   -6.3  -1.7]
 [  0.    0.    0.   -5.    5.6   0.    0.    0.   -1.7   1.9]
 [ 10.7 -11.9   0.    0.    0.   -3.7   4.2   0.    0.    0. ]
 [-11.9   5.    9.2   0.    0.    4.2  -6.5   2.1   0.    0. ]
 [  0.    9.2 -16.5   7.    0.    0.    2.1  -1.4  -1.    0. ]
 [  0.    0.    7.   -6.3  -1.7   0.    0.   -1.   -3.4   5. ]
 [  0.    0.    0.   -1.7   1.9   0.    0.    0.    5.   -5.6]]


Get numerical value of symbolic $A$ matrix from sympy

In [12]:
Nangle_rule = dict(zip(angvec,phi_vals))
NA_mtrx = A_mtrx.xreplace(Nangle_rule)
NA_mtrx = np.array(NA_mtrx,dtype = float)

Matches matrix from resonant interactions?

In [13]:
np.alltrue(np.isclose(NA_mtrx,A))

True

Numerical value for $\mathbf{b}$ vector from sympy

In [14]:
Nb_vec = sp.Matrix(b_vec).xreplace(Nangle_rule)
Nb_vec = np.array(Nb_vec,dtype=float).reshape(-1)

Matches vector from resonant interactions?

In [15]:
np.alltrue(np.isclose(b,Nb_vec))

True